# 1 set up

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 2 import data

In [ ]:
train_dataset = pd.read_csv('../input/loan-prediction-based-on-customer-behavior/Training Data.csv')
print(train_dataset)

# 3 turn words to math, tensorflow,she like math

In [ ]:
train_dataset["Profession"]=pd.factorize(train_dataset.Profession)[0]
train_dataset["CITY"]=pd.factorize(train_dataset.CITY)[0]
train_dataset["STATE"]=pd.factorize(train_dataset.STATE)[0]
train_dataset["Married/Single"]=pd.factorize(train_dataset['Married/Single'])[0]
train_dataset["House_Ownership"]=pd.factorize(train_dataset.House_Ownership)[0]
train_dataset["Car_Ownership"]=pd.factorize(train_dataset.Car_Ownership)[0]


print(train_dataset)

In [ ]:
# plot the number of the deflat and not deflat
count_class=pd.value_counts(train_dataset['Risk_Flag'],sort=True).sort_index()
count_class.plot(kind='bar')
plt.title('Risk_Flag')
plt.xlabel('riks_status')
plt.ylabel('Frequency')
plt.show()


# 4 normlized the income

# 5 make the train_dataset input pre

In [ ]:
# split data
x_train = train_dataset.loc[:,train_dataset.columns!='Risk_Flag']
#标签
y_train = train_dataset.loc[:,train_dataset.columns=='Risk_Flag']
data = train_dataset
input_shape = [x_train.shape[1]]

from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, shuffle= True)

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32) 
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
x_valid = tf.convert_to_tensor(x_valid, dtype=tf.float32) 
y_valid = tf.convert_to_tensor(y_valid, dtype=tf.int32)

# 7 make train_ test_ data turn into tensor

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu', input_shape=input_shape),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(256, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(10, kernel_regularizer=regularizers.l2(0.0001),
                 activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train, epochs=200,validation_data=(x_valid,y_valid),verbose=2)

# modeling

In [ ]:
predictions = model.predict(x_valid)
y_pred = np.argmax(predictions, axis=-1)
print(y_pred)


In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_valid, y_pred)
print(auc)